## Git Stash

Before you can `git pull`, you need to have committed any changes you have made. If you find you want to pull, but you're not ready to commit, you have to temporarily "put aside" your uncommitted changes.
For this, you can use the `git stash` command, like in the following example:

In [1]:
import os
top_dir = os.getcwd()
git_dir = os.path.join(top_dir, 'learning_git')
working_dir = os.path.join(git_dir, 'git_example')
os.chdir(working_dir)

In [2]:
%%writefile Wales.md
Mountains In Wales
==================

* Pen y Fan
* Tryfan
* Snowdon
* Glyder Fawr
* Fan y Big
* Cadair Idris

Overwriting Wales.md


In [3]:
%%bash
git stash
git pull

No local changes to save


From https://github.com/Giovanni1085/github-example
 + 15cdc3c...d852e8e master     -> jamespjh/master  (forced update)
fatal: refusing to merge unrelated histories


By stashing your work first, your repository becomes clean, allowing you to pull. To restore your changes, use `git stash apply`.

In [4]:
%%bash
git stash apply

No stash entries found.


The "Stash" is a way of temporarily saving your working area, and can help out in a pinch.

## Tagging

Tags are easy to read labels for revisions, and can be used anywhere we would name a commit.

Produce real results *only* with tagged revisions

In [5]:
%%bash
git tag --delete v1.0
git tag -a v1.0 -m "Release 1.0"
git push --tags

Deleted tag 'v1.0' (was a074787)


To https://github.com/Giovanni1085/github-example.git
 * [new tag]         v1.0 -> v1.0


In [6]:
%%writefile Pennines.md

Mountains In the Pennines
========================

* Cross Fell

Writing Pennines.md


In [7]:
%%bash
git add Pennines.md
git commit -am "Add Pennines"

[master 2219ad0] Add Pennines
 2 files changed, 7 insertions(+), 1 deletion(-)
 create mode 100644 Pennines.md


You can also use tag names in the place of commmit hashes, such as to list the history between particular commits:

In [8]:
%%bash
git log v1.0.. --graph --oneline

* 2219ad0 Add Pennines


If .. is used without a following commit name, HEAD is assumed.

## Working with generated files: gitignore

We often end up with files that are generated by our program. It is bad practice to keep these in Git; just keep the sources.

Examples include `.o` and `.x` files for compiled languages, `.pyc` files in Python.

In our example, we might want to make our .md files into a PDF with pandoc:

In [9]:
%%writefile Makefile

MDS=$(wildcard *.md)
PDFS=$(MDS:.md=.pdf)

default: $(PDFS)

%.pdf: %.md
	pandoc $< -o $@

Writing Makefile


In [10]:
%%bash
make

pandoc Pennines.md -o Pennines.pdf
pandoc Scotland.md -o Scotland.pdf
pandoc Wales.md -o Wales.pdf
pandoc index.md -o index.pdf
pandoc lakeland.md -o lakeland.pdf


We now have a bunch of output .pdf files corresponding to each Markdown file.

But we don't want those to show up in git:

In [11]:
%%bash
git status

On branch master
Your branch is ahead of 'origin/master' by 2 commits.
Untracked files:
	Makefile
	Pennines.pdf
	Scotland.pdf
	Wales.pdf
	index.pdf
	lakeland.pdf
	wsd.py
	wsd.pyc

nothing added to commit but untracked files present


Use .gitignore files to tell Git not to pay attention to files with certain paths:

In [12]:
%%writefile .gitignore
*.pdf

Writing .gitignore


In [13]:
%%bash
git status

On branch master
Your branch is ahead of 'origin/master' by 2 commits.
Untracked files:
	.gitignore
	Makefile
	wsd.py
	wsd.pyc

nothing added to commit but untracked files present


In [14]:
%%bash
git add Makefile
git add .gitignore
git commit -am "Add a makefile and ignore generated files"
git push

[master 3a03c1c] Add a makefile and ignore generated files
 2 files changed, 9 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 Makefile


To git@github.com:UCL/github-example.git
   5f73fff..3a03c1c  master -> master


## Git clean

Sometimes you end up creating various files that you do not want to include in version control. An easy way of deleting them (if that is what you want) is the `git clean` command, which will remove the files that git is not tracking.

In [15]:
%%bash
git clean -fX

Removing Pennines.pdf
Removing Scotland.pdf
Removing Wales.pdf
Removing index.pdf
Removing lakeland.pdf


In [16]:
%%bash
ls

Makefile
Pennines.md
Scotland.md
Wales.md
index.md
lakeland.md
wsd.py
wsd.pyc


* With -f: don't prompt
* with -d: remove directories
* with -x: Also remote .gitignored files
* with -X: Only remove .gitignore files

## Hunks

### Git Hunks

A "Hunk" is one git change. This changeset has three hunks:

```python
+import matplotlib
+import numpy as np

 from matplotlib import pylab
 from matplotlib.backends.backend_pdf import PdfPages

+def increment_or_add(key,hash,weight=1):
+       if key not in hash:
+               hash[key]=0
+       hash[key]+=weight
+
 data_path=os.path.join(os.path.dirname(
                        os.path.abspath(__file__)),
-regenerate=False
+regenerate=True
```

### Interactive add

`git add` and `git reset` can be used to stage/unstage a whole file,
but you can use interactive mode to stage by hunk, choosing
yes or no for each hunk.

``` bash
git add -p myfile.py
```

``` python
+import matplotlib
+import numpy as np
#Stage this hunk [y,n,a,d,/,j,J,g,e,?]?
```

## GitHub pages

### Yaml Frontmatter

GitHub will publish repositories containing markdown as web pages, automatically. 

You'll need to add this content:

> ```
>    ---
>    ---
> ```

A pair of lines with three dashes, to the top of each markdown file. This is how GitHub knows which markdown files to make into web pages.
[Here's why](https://jekyllrb.com/docs/front-matter/) for the curious. 

In [17]:
%%writefile index.md
---
title: Github Pages Example
---
Mountains and Lakes in the UK
===================

Engerland is not very mountainous.
But has some tall hills, and maybe a mountain or two depending on your definition.

Overwriting index.md


In [18]:
%%bash
git commit -am "Add github pages YAML frontmatter"

[master a19da7c] Add github pages YAML frontmatter
 1 file changed, 6 insertions(+), 2 deletions(-)


### The gh-pages branch

GitHub creates github pages when you use a special named branch.

This is best used to create documentation for a program you write, but you can use it for anything.

In [19]:
os.chdir(working_dir)

In [20]:
%%bash

git checkout -b gh-pages
git push -uf origin gh-pages

Switched to a new branch 'gh-pages'
To git@github.com:UCL/github-example.git
 ! [rejected]        gh-pages -> gh-pages (non-fast-forward)
error: failed to push some refs to 'git@github.com:UCL/github-example.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


The first time you do this, GitHub takes a few minutes to generate your pages. 

The website will appear at `http://username.github.io/repositoryname`, for example:

http://UCL.github.io/github-example/

### UCL layout for GitHub pages

You can use GitHub pages to make HTML layouts, here's an [example of how to do it](http://github.com/UCL/ucl-github-pages-example), 
and [how it looks](http://ucl.github.com/ucl-github-pages-example). We won't go into the detail of this now, 
but after the class, you might want to try this.